In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [4]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('dcgan')):
    os.mkdir('saved_models_mnist_{}'.format('dcgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('dcgan'), mode='w')
import os
import tensorflow as tf

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % save_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=30, batch_size=64, save_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

epoch:0 step:56 [D loss: 0.793782, acc.: 54.69%] [G loss: 1.136590]
epoch:0 step:57 [D loss: 0.900310, acc.: 49.22%] [G loss: 1.087780]
epoch:0 step:58 [D loss: 0.974866, acc.: 42.19%] [G loss: 1.188643]
epoch:0 step:59 [D loss: 0.800961, acc.: 49.22%] [G loss: 1.388608]
epoch:0 step:60 [D loss: 1.013465, acc.: 42.97%] [G loss: 1.213990]
epoch:0 step:61 [D loss: 0.865639, acc.: 45.31%] [G loss: 1.268679]
epoch:0 step:62 [D loss: 0.893454, acc.: 46.09%] [G loss: 0.991829]
epoch:0 step:63 [D loss: 0.780858, acc.: 51.56%] [G loss: 1.332237]
epoch:0 step:64 [D loss: 0.874093, acc.: 45.31%] [G loss: 1.238858]
epoch:0 step:65 [D loss: 0.845127, acc.: 47.66%] [G loss: 1.088136]
epoch:0 step:66 [D loss: 0.845404, acc.: 51.56%] [G loss: 1.069645]
epoch:0 step:67 [D loss: 0.776089, acc.: 53.91%] [G loss: 1.079013]
epoch:0 step:68 [D loss: 0.823574, acc.: 46.88%] [G loss: 1.359123]
epoch:0 step:69 [D loss: 0.915007, acc.: 43.75%] [G loss: 1.346519]
epoch:0 step:70 [D loss: 0.813153, acc.: 51.56%]

epoch:0 step:179 [D loss: 0.879297, acc.: 39.06%] [G loss: 1.206634]
epoch:0 step:180 [D loss: 1.012597, acc.: 38.28%] [G loss: 1.074492]
epoch:0 step:181 [D loss: 0.823548, acc.: 43.75%] [G loss: 1.181467]
epoch:0 step:182 [D loss: 0.868376, acc.: 41.41%] [G loss: 1.057033]
epoch:0 step:183 [D loss: 0.870114, acc.: 39.84%] [G loss: 0.955756]
epoch:0 step:184 [D loss: 0.831612, acc.: 43.75%] [G loss: 0.936626]
epoch:0 step:185 [D loss: 0.798696, acc.: 50.00%] [G loss: 1.013512]
epoch:0 step:186 [D loss: 0.789430, acc.: 48.44%] [G loss: 1.017732]
epoch:0 step:187 [D loss: 0.750968, acc.: 57.03%] [G loss: 0.997275]
epoch:0 step:188 [D loss: 0.968811, acc.: 34.38%] [G loss: 1.008533]
epoch:0 step:189 [D loss: 0.795639, acc.: 50.78%] [G loss: 0.974146]
epoch:0 step:190 [D loss: 0.851629, acc.: 46.88%] [G loss: 1.038645]
epoch:0 step:191 [D loss: 0.824703, acc.: 48.44%] [G loss: 1.184436]
epoch:0 step:192 [D loss: 0.807741, acc.: 46.88%] [G loss: 1.189656]
epoch:0 step:193 [D loss: 0.834006

epoch:0 step:292 [D loss: 0.680974, acc.: 62.50%] [G loss: 1.124110]
epoch:0 step:293 [D loss: 0.748121, acc.: 46.88%] [G loss: 0.988115]
epoch:0 step:294 [D loss: 0.771419, acc.: 50.00%] [G loss: 1.129985]
epoch:0 step:295 [D loss: 0.772048, acc.: 51.56%] [G loss: 0.978776]
epoch:0 step:296 [D loss: 0.731627, acc.: 52.34%] [G loss: 1.027727]
epoch:0 step:297 [D loss: 0.785997, acc.: 43.75%] [G loss: 0.951421]
epoch:0 step:298 [D loss: 0.749509, acc.: 53.12%] [G loss: 1.138070]
epoch:0 step:299 [D loss: 0.746011, acc.: 50.78%] [G loss: 1.106012]
epoch:0 step:300 [D loss: 0.763755, acc.: 56.25%] [G loss: 1.039539]
epoch:0 step:301 [D loss: 0.901095, acc.: 39.06%] [G loss: 0.941761]
epoch:0 step:302 [D loss: 0.746578, acc.: 53.91%] [G loss: 1.003380]
epoch:0 step:303 [D loss: 0.807285, acc.: 50.78%] [G loss: 0.993046]
epoch:0 step:304 [D loss: 0.796305, acc.: 52.34%] [G loss: 0.974527]
epoch:0 step:305 [D loss: 0.754809, acc.: 50.00%] [G loss: 0.820766]
epoch:0 step:306 [D loss: 0.866145

epoch:0 step:409 [D loss: 0.824496, acc.: 46.09%] [G loss: 1.093808]
epoch:0 step:410 [D loss: 0.795154, acc.: 47.66%] [G loss: 1.092709]
epoch:0 step:411 [D loss: 0.811240, acc.: 46.09%] [G loss: 1.080116]
epoch:0 step:412 [D loss: 0.765280, acc.: 52.34%] [G loss: 1.045914]
epoch:0 step:413 [D loss: 0.741008, acc.: 50.00%] [G loss: 1.031773]
epoch:0 step:414 [D loss: 0.718126, acc.: 55.47%] [G loss: 1.003462]
epoch:0 step:415 [D loss: 0.684526, acc.: 57.81%] [G loss: 0.961490]
epoch:0 step:416 [D loss: 0.826328, acc.: 48.44%] [G loss: 1.002316]
epoch:0 step:417 [D loss: 0.824065, acc.: 50.78%] [G loss: 1.026494]
epoch:0 step:418 [D loss: 0.746421, acc.: 49.22%] [G loss: 1.059024]
epoch:0 step:419 [D loss: 0.761283, acc.: 56.25%] [G loss: 0.957641]
epoch:0 step:420 [D loss: 0.625644, acc.: 62.50%] [G loss: 1.184703]
epoch:0 step:421 [D loss: 0.778185, acc.: 51.56%] [G loss: 0.983758]
epoch:0 step:422 [D loss: 0.751223, acc.: 47.66%] [G loss: 1.048155]
epoch:0 step:423 [D loss: 0.761098

...

epoch:29 step:27590 [D loss: 0.901648, acc.: 38.28%] [G loss: 0.857913]
epoch:29 step:27591 [D loss: 0.820829, acc.: 49.22%] [G loss: 0.855266]
epoch:29 step:27592 [D loss: 0.421525, acc.: 84.38%] [G loss: 1.789577]
epoch:29 step:27593 [D loss: 0.601447, acc.: 70.31%] [G loss: 1.153553]
epoch:29 step:27594 [D loss: 1.028463, acc.: 29.69%] [G loss: 0.958121]
epoch:29 step:27595 [D loss: 0.743399, acc.: 52.34%] [G loss: 1.424889]
epoch:29 step:27596 [D loss: 0.899000, acc.: 39.84%] [G loss: 1.100794]
epoch:29 step:27597 [D loss: 0.547735, acc.: 72.66%] [G loss: 1.033480]
epoch:29 step:27598 [D loss: 0.583493, acc.: 70.31%] [G loss: 1.066093]
epoch:29 step:27599 [D loss: 0.722107, acc.: 57.81%] [G loss: 1.117396]
epoch:29 step:27600 [D loss: 0.757311, acc.: 48.44%] [G loss: 1.272599]
##############
[2.41884833 1.75907574 5.81465574 4.62882448 3.04902291 5.4051314
 4.03679221 4.41806039 4.19556183 3.34749855]
##########
epoch:29 step:27601 [D loss: 0.577601, acc.: 69.53%] [G loss: 1.353707

epoch:29 step:27704 [D loss: 0.674462, acc.: 57.03%] [G loss: 1.430772]
epoch:29 step:27705 [D loss: 0.410679, acc.: 83.59%] [G loss: 1.669879]
epoch:29 step:27706 [D loss: 0.307411, acc.: 92.97%] [G loss: 1.445232]
epoch:29 step:27707 [D loss: 0.434199, acc.: 82.03%] [G loss: 1.260602]
epoch:29 step:27708 [D loss: 0.489463, acc.: 78.12%] [G loss: 1.085173]
epoch:29 step:27709 [D loss: 0.340404, acc.: 89.84%] [G loss: 1.479182]
epoch:29 step:27710 [D loss: 0.267321, acc.: 92.97%] [G loss: 1.588924]
epoch:29 step:27711 [D loss: 0.586005, acc.: 70.31%] [G loss: 1.605783]
epoch:29 step:27712 [D loss: 0.578019, acc.: 73.44%] [G loss: 1.449624]
epoch:29 step:27713 [D loss: 0.612567, acc.: 67.19%] [G loss: 1.034666]
epoch:29 step:27714 [D loss: 0.607586, acc.: 70.31%] [G loss: 1.034946]
epoch:29 step:27715 [D loss: 0.781639, acc.: 47.66%] [G loss: 0.849014]
epoch:29 step:27716 [D loss: 0.444088, acc.: 86.72%] [G loss: 1.360644]
epoch:29 step:27717 [D loss: 0.474105, acc.: 82.81%] [G loss: 1.

epoch:29 step:27817 [D loss: 0.690202, acc.: 60.94%] [G loss: 1.277336]
epoch:29 step:27818 [D loss: 0.564049, acc.: 74.22%] [G loss: 1.007800]
epoch:29 step:27819 [D loss: 0.399687, acc.: 83.59%] [G loss: 1.306657]
epoch:29 step:27820 [D loss: 0.343507, acc.: 89.84%] [G loss: 1.288900]
epoch:29 step:27821 [D loss: 0.650845, acc.: 60.16%] [G loss: 1.035024]
epoch:29 step:27822 [D loss: 0.407113, acc.: 83.59%] [G loss: 1.517120]
epoch:29 step:27823 [D loss: 0.444322, acc.: 82.81%] [G loss: 1.488487]
epoch:29 step:27824 [D loss: 0.608612, acc.: 67.19%] [G loss: 1.123974]
epoch:29 step:27825 [D loss: 0.680516, acc.: 60.16%] [G loss: 1.104404]
epoch:29 step:27826 [D loss: 0.444631, acc.: 85.94%] [G loss: 1.386971]
epoch:29 step:27827 [D loss: 0.461872, acc.: 83.59%] [G loss: 1.149473]
epoch:29 step:27828 [D loss: 0.583396, acc.: 68.75%] [G loss: 1.243169]
epoch:29 step:27829 [D loss: 0.567115, acc.: 72.66%] [G loss: 1.138445]
epoch:29 step:27830 [D loss: 0.583514, acc.: 71.09%] [G loss: 1.

epoch:29 step:27934 [D loss: 0.663584, acc.: 59.38%] [G loss: 1.246855]
epoch:29 step:27935 [D loss: 0.666421, acc.: 65.62%] [G loss: 0.998058]
epoch:29 step:27936 [D loss: 0.508755, acc.: 73.44%] [G loss: 0.891932]
epoch:29 step:27937 [D loss: 0.648696, acc.: 60.16%] [G loss: 1.189983]
epoch:29 step:27938 [D loss: 0.763799, acc.: 53.12%] [G loss: 1.612257]
epoch:29 step:27939 [D loss: 0.621532, acc.: 68.75%] [G loss: 1.352158]
epoch:29 step:27940 [D loss: 0.376905, acc.: 89.84%] [G loss: 1.730032]
epoch:29 step:27941 [D loss: 0.245180, acc.: 95.31%] [G loss: 1.564081]
epoch:29 step:27942 [D loss: 0.291458, acc.: 89.06%] [G loss: 1.740634]
epoch:29 step:27943 [D loss: 0.337901, acc.: 92.19%] [G loss: 1.775219]
epoch:29 step:27944 [D loss: 0.762934, acc.: 54.69%] [G loss: 1.325857]
epoch:29 step:27945 [D loss: 0.775800, acc.: 49.22%] [G loss: 0.917859]
epoch:29 step:27946 [D loss: 0.345928, acc.: 94.53%] [G loss: 1.426506]
epoch:29 step:27947 [D loss: 0.267536, acc.: 90.62%] [G loss: 1.

epoch:29 step:28050 [D loss: 0.888938, acc.: 39.06%] [G loss: 0.860775]
epoch:29 step:28051 [D loss: 0.765020, acc.: 50.78%] [G loss: 1.057114]
epoch:29 step:28052 [D loss: 0.633097, acc.: 63.28%] [G loss: 1.774445]
epoch:29 step:28053 [D loss: 0.523498, acc.: 78.12%] [G loss: 1.061970]
epoch:29 step:28054 [D loss: 0.600184, acc.: 66.41%] [G loss: 1.144270]
epoch:29 step:28055 [D loss: 0.583296, acc.: 67.97%] [G loss: 0.783142]
epoch:29 step:28056 [D loss: 0.406513, acc.: 85.94%] [G loss: 1.533758]
epoch:29 step:28057 [D loss: 0.429918, acc.: 78.12%] [G loss: 1.450818]
epoch:29 step:28058 [D loss: 0.312818, acc.: 89.84%] [G loss: 1.656180]
epoch:29 step:28059 [D loss: 0.481366, acc.: 78.91%] [G loss: 1.757542]
epoch:29 step:28060 [D loss: 0.300144, acc.: 93.75%] [G loss: 1.636974]
epoch:29 step:28061 [D loss: 0.785990, acc.: 45.31%] [G loss: 1.204800]
epoch:29 step:28062 [D loss: 0.416392, acc.: 82.03%] [G loss: 1.480414]
epoch:29 step:28063 [D loss: 0.674656, acc.: 62.50%] [G loss: 1.